In [1]:
import os
# save using vs code
SCALA_VERSION = '2.12'
SPARK_VERSION = '3.1.3'
# Download Kafka Jar file, this for readStream.format("kafka"), "kafka" is a driver
# kafka driver code is part of Maven Jar file
# https://mvnrepository.com/artifact/org.apache.spark/spark-sql-kafka-0-10_2.12/3.1.3
# pyspark-shell shall download the jar file behind..
#os.environ['PYSPARK_SUBMIT_ARGS'] = f'--packages org.apache.spark:spark-sql-kafka-0-10_{SCALA_VERSION}:{SPARK_VERSION} pyspark-shell'

In [2]:
# here we implement windowed candle data for 1 minute
# here we implement windowed candle data for 3 minute
# here we implement windowed candle data for 5 minute

# we use hadoop-vm as hostname, which is running on another system


#     kafka-topics  --create --bootstrap-server hadoop-vm:9092 --replication-factor 1 --partitions 1 --topic stock-ticks 
    
#     kafka-topics  --create --bootstrap-server hadoop-vm:9092 --replication-factor 1 --partitions 1 --topic candles-1min 
#     kafka-console-consumer --bootstrap-server hadoop-vm:9092 --topic  candles-1min  --from-beginning
 
#     kafka-topics  --create --bootstrap-server hadoop-vm:9092 --replication-factor 1 --partitions 1 --topic candles-3min 
#     kafka-console-consumer --bootstrap-server hadoop-vm:9092 --topic  candles-3min  --from-beginning

#     kafka-topics  --create --bootstrap-server hadoop-vm:9092 --replication-factor 1 --partitions 1 --topic candles-5min 
#     kafka-console-consumer --bootstrap-server hadoop-vm:9092 --topic  candles-5min  --from-beginning

In [3]:
import findspark
findspark.init()

In [4]:
import pyspark


from pyspark.sql import SparkSession
# spark groupBy has default setting for spark.sql.shuffle.partitions as 200
# we set to  4, should NOT be done in production 
spark = SparkSession.builder.master("local[1]")\
                            .config('spark.sql.shuffle.partitions', 4)\
                            .appName("SparkStreamingKafkaCandles").getOrCreate()

22/03/22 00:46:44 WARN Utils: Your hostname, ubuntu-virtual-machine resolves to a loopback address: 127.0.1.1; using 192.168.80.128 instead (on interface ens33)
22/03/22 00:46:44 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/opt/spark-3.1.3-bin-hadoop2.7/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/ubuntu/.ivy2/cache
The jars for the packages stored in: /home/ubuntu/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-0a88a7c3-c6d7-4468-b0eb-76b0e1d4aa1d;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.1.3 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.1.3 in central
	found org.apache.kafka#kafka-clients;2.6.0 in central
	found com.github.luben#zstd-jni;1.4.8-1 in central
	found org.lz4#lz4-java;1.7.1 in central
	found org.xerial.snappy#snappy-java;1.1.8.2 in central
	found org.slf4j#slf4j-api;1.7.30 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found org.apache.commons#commons-pool2;2.6.2 in central
:: resolution report :: resolve 813ms :: artifacts dl 23ms
	:: modules in use:
	com.github.luben#zstd-jni;1.4.8-1 from central in [default]
	org.apache.commons#commons-pool2;2.6.2 from central

In [5]:
kafkaDf = spark.readStream.format("kafka")\
  .option("kafka.bootstrap.servers", "hadoop-vm:9092")\
  .option("subscribe", "stock-ticks")\
  .option("group.id", "stock-ticks-group-JSCtask2")\
  .load()

In [6]:
kafkaDf.printSchema()

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



In [7]:
ticksDf = kafkaDf.selectExpr("CAST(value AS STRING)", "timestamp")
ticksDf.printSchema()

root
 |-- value: string (nullable = true)
 |-- timestamp: timestamp (nullable = true)



In [8]:
import pyspark.sql.functions as F
from pyspark.sql.types import StructField, StructType, DoubleType, StringType, LongType, TimestampType

schema = StructType( [
    StructField("symbol", StringType(), True),
    StructField("price", DoubleType(), True),
    StructField("volume", LongType(), True),
    StructField("timestamp", LongType(),  True)
])

In [9]:
# we parse value which is JSON string to kafka struct, 
# this create a new column [replacing existing value which is string] with type
jsonDf = ticksDf.withColumn("value", F.from_json("value", schema))
jsonDf.printSchema()

root
 |-- value: struct (nullable = true)
 |    |-- symbol: string (nullable = true)
 |    |-- price: double (nullable = true)
 |    |-- volume: long (nullable = true)
 |    |-- timestamp: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)



In [10]:
# now extract all the sub fields symbol, price, volume, timestamp into data frame
# exisitng value, timestamp shall be removed. existing timestamp is value produced time, not a stock tick time
# value.timestamp that comes from producer is basically a event time
stockTickDf = jsonDf.select (F.col("value.*")) # extract all sub fields as schema
stockTickDf.printSchema()

root
 |-- symbol: string (nullable = true)
 |-- price: double (nullable = true)
 |-- volume: long (nullable = true)
 |-- timestamp: long (nullable = true)



In [11]:
stockTickDf = stockTickDf.withColumn("traded_value", F.col("price") * F.col("volume"))
stockTickDf.printSchema()

root
 |-- symbol: string (nullable = true)
 |-- price: double (nullable = true)
 |-- volume: long (nullable = true)
 |-- timestamp: long (nullable = true)
 |-- traded_value: double (nullable = true)



In [13]:
# we have milli seconds 1647283268253, the actual seconds Mon Mar 14 2022 14:41:08
# now we have to calculate 1 minute window, sum(volume) up to 1 minute
# after conversions, we take timestamp which is truncatd to minute instead of milli seconds, drop temp columns
stockTickDf = stockTickDf\
                .withColumn("timestampTemp", (F.col("timestamp") / 1000).cast("timestamp"))\
                .withColumn("trade_time", F.date_trunc("minute", F.col("timestampTemp")))\
                .drop("timestamp")\
                .drop("timestampTemp")\
                .withColumnRenamed("trade_time", "timestamp")

In [ ]:
def processBatchData(candleBatchDf, batch_id):
    print ("process batch called", batch_id, "writing ", candleBatchDf.count())
    # result = candleBatchDf.collect()
    # print(result)
    # candleBatchDf.show(2)
    # write to csv/json/parqeut/database /jdbc etc
    # window is nested column, has its children columns called start and end
    # start and end are part of nested window column
    # append mode will not delete existin data, instead append to existing table
    # do the partition, then write the data
    (
     candleBatchDf
        .coalesce(1)
        .write
        .outputMode("append")
        .format("csv")
        .option("header", True)
         .save( "hdfs://localhost:9000/dump")
    )
    
stockTickDf.writeStream.outputMode("append").foreachBatch(processBatchData).start()

In [13]:
import pyspark.sql.functions as F
# output DataStreamWriter
# copy symbol to _symbol for partition, as spark shall remove the column 
# if we use  the column in partitionBy

(
    stockTickDf
     .withColumn("year", F.date_format(F.col("timestamp"), "yyyy"))
     .withColumn("month", F.date_format(F.col("timestamp"), "MM"))
     .withColumn("day", F.date_format(F.col("timestamp"), "dd"))  
     .withColumn("hour", F.date_format(F.col("timestamp"), "HH"))   
     .withColumn("_symbol", F.col("symbol"))   
    .writeStream
    .trigger(processingTime='65 seconds')\
    .queryName("Write Ticks to CSV trigger by 1 min hour")
    .format("csv")
    .option("path", "hdfs://localhost:9000/dump-csv-trigger-hourly-1min")
    .option("header", True)
    .option("checkpointLocation", "hdfs://localhost:9000/checkpoint/tickscsvtohdfs6")
    .partitionBy("year", "month", "day", "hour", "_symbol")
    .option("truncate", False)
    .start()
)